# BASIC MODEL

## Classes

In [1]:
#import libraries
import numpy as np
import pandas as pd
import pylab as plt
import time
import glob, os, os.path
import osmnx as ox
import networkx as nx
import cv2

In [34]:
class Vehicle:
  """A vehicle class"""
  count = 0 
  vehicles = []
  def __init__(self,verb=False):
    self.v_uid = self.count #unique id
    self.v_home = (0,0) #initial location
    self.v_dead = False #to check if it is out of the model
    self.v_state = 0 #code for state=> e.g 0:wait; 1:moving
    self.v_pos = (0,0)
    self.v_origin = (0,0) #start location
    self.v_destination = (0,0) #end location
    self.v_velocity = (0,0) #needs to be adjusted based on unit of model
    self.v_passengers = []
    #to control uid and an aggregated list
    self.__class__.count += 1
    Vehicle.vehicles.append(self)


In [35]:
class Pedestrian:
  """A pedestrian class"""
  count = 0
  peds = []
  def __init__(self,verb=False):
    self.p_uid = self.count
    self.p_age = 0
    self.p_gender = 0 #0:female;1:male
    self.p_home = (0,0)
    self.p_state = 0
    self.p_pos = (0,0)
    self.p_origin = (0,0)
    self.p_destination = (0,0)
    self.p_velocity = (0,0)
    self.p_show = 1
    #to control uid and an aggregated list
    self.__class__.count +=1
    Pedestrian.peds.append(self)

In [36]:
class Environment:
  """A road network environment class"""
  def __init__(self,bbox,ntype='drive',verb=False):
    #bbox is a dict of north,south,east,west lat,lon edges of target area
    self.e_bbox = {'north': bbox['north'], 'south': bbox['south'],
                     'east': bbox['east'], 'west': bbox['west']}
    self.e_network(ntype=ntype,verb=verb)

  def e_network(self,ntype='drive',verb=False):
    # Obtain the roadmap data from OpenStreetMap by using OSMNX 
    self.e_G = ox.graph_from_bbox(self.e_bbox['north'], self.e_bbox['south'], 
                           self.e_bbox['east'], self.e_bbox['west'], 
                           network_type=ntype)
  
  def e_project_network(self,verb=False):
    #Project network
    self.e_Gp = ox.project_graph(self.e_G)
  
  def e_get_nodes(self,verb=False):
    self.e_nodes, self.e_edges = ox.graph_to_gdfs(self.e_G)
    return self.e_nodes
  
  def e_get_edges(self,verb=False):
    self.e_nodes, self.e_edges = ox.graph_to_gdfs(self.e_G)
    return self.e_edges

  def e_plot(self,verb=False):
    #returning a fig and ax
    return ox.plot_graph(self.e_G,bgcolor='w',node_color='k',figsize=(16,8),
                         node_alpha=0.1,edge_color=(0,0,0,0.5),show=False,
                         close=False)

In [37]:
class Model:
  "A model class"
  def __init__(self,bbox=None,ntype='drive',vehicles=10,
               population=50,verb=False):
    self.Env = Environment(bbox,ntype=ntype,verb=verb)
    self.nodes = self.Env.e_get_nodes()
    self.edges = self.Env.e_edges
    for i in range(vehicles):
      v = Vehicle()
      s = self.nodes.sample()
      v.v_home = (float(s['x']),
                float(s['y']))
    for i in range(population):
      p = Pedestrian()
      s = self.nodes.sample()
      p.p_home = (float(s['x']),
                float(s['y']))

  def go(self,sim_time=5,plot=False,video=False,verb=False):
    for t in range(sim_time):
      for i,v in enumerate(Vehicle.vehicles):
        s = self.nodes.sample()
        v.v_home = (float(s['x']),
                float(s['y']))
      if plot:
        self.plot(id=t,save=True,show=False)
    if video:
      self.video()

  def plot(self,id=0,save=False,show=False,verb=False):
    #plot network
    fig,ax = self.Env.e_plot()
    fig.tight_layout()
    #plot vehicles
    x = [v.v_home[0] for v in Vehicle.vehicles]
    y = [v.v_home[1] for v in Vehicle.vehicles]
    ax.scatter(x,y,c='b',marker='s')
    #plot pedestrians
    x = [p.p_home[0] for p in Pedestrian.peds]
    y = [p.p_home[1] for p in Pedestrian.peds]
    ax.scatter(x,y,c='m',marker='.')
    if save:
      plt.savefig(f'./img/fig{id:04d}.png')
      plt.close()
      print(f'figure {id:04d} plotted')
    if show:
      plt.show()
  
  def video(self,image_folder='./img',
            video_name = './anim.mp4',
            fps=1,verbose=False):
    images = [img for img in sorted(os.listdir(image_folder)) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name,cv2.VideoWriter_fourcc(*'MP4V'),#('M','J','P','G'), 
                            fps, (width,height))
    
    for image in images:
        if verbose:
            print(image)
        video.write(cv2.imread(os.path.join(image_folder, image)))
    cv2.destroyAllWindows()
    video.release()

## Parameters

For bounding box (bbox) of other areas:
[OSM](https://www.openstreetmap.org/export#map=5/33.907/138.460)

In [38]:
#BBOX
#small area is faster (by default this is in the class)
arahama = {'north': 38.2271, 'south': 38.2077,
           'east': 140.9894, 'west': 140.9695}

#this is the same extension from Abe san's simulation
kochi = {'north': 33.5978428707312631, 'south': 33.3844862625877710,
           'east': 133.7029719124942346, 'west': 133.3254475395832799}

## Running the model

In [43]:
%%time
#create a model class of an area with number of vehicles and population
M = Model(bbox=kochi,ntype='walk',vehicles=10,population=50,verb=False)

CPU times: user 57.5 s, sys: 1.31 s, total: 58.8 s
Wall time: 1min 16s


In [44]:
%%time
#plot current situation ('id' is an integer)
M.plot(id=1,save=True,show=True)

figure 0001 plotted
CPU times: user 20.4 s, sys: 432 ms, total: 20.9 s
Wall time: 23.4 s


In [52]:
%%time
#number of steps for simulation, can plot and make video same time
M.go(sim_time=100,plot=True,video=True)

figure 0000 plotted
figure 0001 plotted
figure 0002 plotted
figure 0003 plotted
figure 0004 plotted
figure 0005 plotted
figure 0006 plotted
figure 0007 plotted
figure 0008 plotted
figure 0009 plotted
figure 0010 plotted
figure 0011 plotted
figure 0012 plotted
figure 0013 plotted
figure 0014 plotted
figure 0015 plotted
figure 0016 plotted
figure 0017 plotted
figure 0018 plotted
figure 0019 plotted
figure 0020 plotted
figure 0021 plotted
figure 0022 plotted
figure 0023 plotted
figure 0024 plotted
figure 0025 plotted
figure 0026 plotted
figure 0027 plotted
figure 0028 plotted
figure 0029 plotted
figure 0030 plotted
figure 0031 plotted
figure 0032 plotted
figure 0033 plotted
figure 0034 plotted
figure 0035 plotted
figure 0036 plotted
figure 0037 plotted
figure 0038 plotted
figure 0039 plotted
figure 0040 plotted
figure 0041 plotted
figure 0042 plotted
figure 0043 plotted
figure 0044 plotted
figure 0045 plotted
figure 0046 plotted
figure 0047 plotted
figure 0048 plotted
figure 0049 plotted


In [65]:
%%time
#only to make video
M.video()

CPU times: user 841 ms, sys: 42.7 ms, total: 884 ms
Wall time: 920 ms


# Overlaying the inundation data

In [ ]:
#this is the same extension from Abe san's simulation
kochi = {'north': 33.5978428707312631, 'south': 33.3844862625877710,
           'east': 133.7029719124942346, 'west': 133.3254475395832799}

e = Environment(kochi)

In [ ]:
fig = e.e_plot()

In [ ]:
nodes = e.e_get_nodes()
edges = e.e_get_edges()

In [ ]:
import rasterio
from rasterio.plot import show

In [ ]:
raster = rasterio.open('./max5_wgs84_kochishi.tif')

In [ ]:
tsu = []
for row in nodes.iterrows():
    x = row[1].x
    y = row[1].y
    row, col = raster.index(x,y)
    try:
        tsu.append(raster.read(1)[row,col])
    except:
        tsu.append(-99)

In [ ]:
for i,v in enumerate(tsu):
    if v == -99.:
        tsu[i]=0

In [ ]:
max(tsu)

In [ ]:
nodes['tsu']=tsu

In [ ]:
nx.set_node_attributes(e.e_G, values=nodes['tsu'], name="depth")
nc = ox.plot.get_node_colors_by_attr(e.e_G, 'depth', cmap='Blues', start=0, stop=1, na_color='none', equal_size=True)
ncr = ox.plot.get_node_colors_by_attr(e.e_G, 'depth', cmap='Reds', start=0, stop=1, na_color='none', equal_size=False)

In [ ]:
import matplotlib as mpl
import matplotlib.cm as cm
cmap = plt.cm.get_cmap('Blues')
norm=plt.Normalize(vmin=nodes['tsu'].min(), vmax=nodes['tsu'].max())
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])

fig,ax = ox.plot_graph(e.e_G,figsize=(16,8),bgcolor='w',node_color=nc,
                         node_alpha=0.8,node_edgecolor='grey',edge_color='grey',show=False,
                         close=False)
fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax, orientation='vertical')
fig.savefig('inundation.png')
                    

In [ ]:
nodes

In [ ]:
edges

In [ ]:
# edges['risk']=0.5*(nodes.loc[edges['u']]['tsu']+nodes.loc[edges['v']]['tsu'])
risk=[]
for row in edges.iterrows():
    risk.append(0.5*(nodes.loc[row[1].u]['tsu']+nodes.loc[row[1].v]['tsu']))

In [ ]:
edges['risk']=risk

In [ ]:
p = ox.shortest_path(e.e_G, 253768914, 8721376111, weight='risk')
ox.plot.plot_graph_route(e.e_G, p, route_color='r', route_linewidth=2, route_alpha=0.5, orig_dest_size=100, ax=ax)
fig.savefig('route.png')